In [1]:
import mxnet as mx
from mxnet import gluon, autograd, ndarray
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

In [2]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

In [3]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target',1), df['target'],
                                                    test_size=0.2, random_state=1111)

### Gluon Model

In [5]:
BATCH_SIZE = 32
LEARNING_R = 0.001
EPOCHS = 300

### Prepare data 

In [6]:
train_dataset = mx.gluon.data.ArrayDataset(X_train.as_matrix(),y_train.as_matrix())
test_dataset = mx.gluon.data.ArrayDataset(X_test.as_matrix(),y_test.as_matrix())


In [7]:
train_data = mx.gluon.data.DataLoader(train_dataset,
                                      batch_size=BATCH_SIZE, shuffle=True)

test_data = mx.gluon.data.DataLoader(test_dataset,
                                     batch_size=BATCH_SIZE, shuffle=False)

### Initialize the model

In [13]:
net = gluon.nn.Sequential()

# Define the model architecture
with net.name_scope():
    net.add(gluon.nn.Dense(64, activation="relu"))
    #net.add(gluon.nn.Dropout(0.8)) 
    net.add(gluon.nn.Dense(32, activation="relu") ) 
    net.add(gluon.nn.BatchNorm())    
    net.add(gluon.nn.Dense(1,  activation="sigmoid"))

# Intitalize parametes of the model
net.collect_params().initialize(mx.init.Uniform())

# Add binary loss function
binary_cross_entropy = gluon.loss.SigmoidBinaryCrossEntropyLoss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': LEARNING_R})


In [14]:
for e in range(EPOCHS):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(mx.cpu()).astype('float32')
        label = label.as_in_context(mx.cpu()).astype('float32')
        with autograd.record(): # Start recording the derivatives
            output = net(data) # the forward iteration
            loss = binary_cross_entropy(output, label)
            loss.backward()
        trainer.step(data.shape[0])
        # Provide stats on the improvement of the model over each epoch
        curr_loss = ndarray.mean(loss).asscalar()
    if e % 20 == 0:
        print("Epoch {}. Current Loss: {}.".format(e, curr_loss))

Epoch 0. Current Loss: 0.5764642953872681.
Epoch 20. Current Loss: 0.4895660877227783.
Epoch 40. Current Loss: 0.7460066080093384.
Epoch 60. Current Loss: 0.47263389825820923.
Epoch 80. Current Loss: 0.5592411160469055.
Epoch 100. Current Loss: 0.5979680418968201.
Epoch 120. Current Loss: 0.578061044216156.
Epoch 140. Current Loss: 0.4670383930206299.
Epoch 160. Current Loss: 0.6739763617515564.
Epoch 180. Current Loss: 0.4955267012119293.
Epoch 200. Current Loss: 0.4690001308917999.
Epoch 220. Current Loss: 0.45677265524864197.
Epoch 240. Current Loss: 0.6150979995727539.
Epoch 260. Current Loss: 0.509056031703949.
Epoch 280. Current Loss: 0.5234965682029724.


### Predict

In [15]:
y_pred = np.array([])
for data,label in test_data:
        data = data.as_in_context(mx.cpu()).astype('float32')
        label = label.as_in_context(mx.cpu()).astype('float32')
        output = net(data)
        y_pred = np.append(y_pred, output.asnumpy())

In [16]:
y_pred_labels = np.where(y_pred > 0.48, 1, 0)

In [12]:
accuracy_score(y_test, y_pred_labels)

0.91350877192982459